In [1]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.4.0
Torchvision Version:  0.5.0


In [2]:
import torchvision.models as models

In [3]:
vgg16 = models.vgg16(pretrained=True, progress=True)
# this will download the entire model of 536 MB, as it has 138 million learnable params

In [34]:

class VGG_FEATURES(nn.Module):
    def __init__(self, original_model):
        super(VGG_FEATURES, self).__init__()
        self.features = original_model.features
        self.avgpool = original_model.avgpool
        self.classifier = nn.Sequential(*list(original_model.classifier.children())[:-6])
        
        for param in self.features.parameters():
            param.requires_grad = False
        
        for param in self.classifier.parameters():
            param.requires_grad = False
        
        for param in self.avgpool.parameters():
            param.requires_grad = False
        
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x
        

In [35]:
vgg = VGG_FEATURES(vgg16)



In [6]:
from torchvision import transforms
from PIL import Image

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])g

img_path = 'dataset/train/videoclips/clip_1/000017.jpg'
input_image = Image.open(img_path)
input_tensor = preprocess(input_image)
input_tensor.shape
input_batch = input_tensor.unsqueeze(0)
op = vgg16(input_batch)

In [7]:
op = vgg16(input_batch)

In [8]:
print(op.shape)

torch.Size([1, 1000])


In [36]:
fe = vgg(input_batch)

In [37]:
fe=  fe

In [38]:
fe

tensor([[ 1.3240, -0.5318,  2.9833,  ..., -3.5269, -1.9972, -0.5391]])

In [22]:
torch.save(vgg, './trained_models/vgg.model')

//anaconda3/lib/python3.7/site-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type VGG_FEATURES. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [14]:
!mkdir trained_models

In [23]:
loaded_model = torch.load('./trained_models/vgg.model')

In [24]:
loaded_model(input_batch).flatten().shape

torch.Size([4096])